In [3]:
'''This is a reproduction of the IRNN experiment
with pixel-by-pixel sequential MNIST in
"A Simple Way to Initialize Recurrent Networks of Rectified Linear Units"
by Quoc V. Le, Navdeep Jaitly, Geoffrey E. Hinton
arxiv:1504.00941v2 [cs.NE] 7 Apr 2015
http://arxiv.org/pdf/1504.00941v2.pdf
Optimizer is replaced with RMSprop which yields more stable and steady
improvement.
Reaches 0.93 train/test accuracy after 900 epochs
(which roughly corresponds to 1687500 steps in the original paper.)
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras import initializers
from keras.optimizers import RMSprop


In [4]:

batch_size = 32
num_classes = 10
epochs = 200
hidden_units = 100

learning_rate = 1e-6
clip_norm = 1.0

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], -1, 1)
x_test = x_test.reshape(x_test.shape[0], -1, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



x_train shape: (60000, 784, 1)
60000 train samples
10000 test samples


In [5]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)

print("x_test", x_test.shape)
print("y_test", y_test.shape)

print("hidden_units", hidden_units)
print("input_shape", x_train.shape[1:])

x_train (60000, 784, 1)
y_train (60000, 10)
x_test (10000, 784, 1)
y_test (10000, 10)
hidden_units 100
input_shape (784, 1)


In [6]:
input_shape = x_train.shape[1:]
print('Evaluate IRNN...')
model = Sequential()
model.add(SimpleRNN(hidden_units,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=input_shape))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])


print(model.summary())

Evaluate IRNN...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 100)               10200     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 11,210
Trainable params: 11,210
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

scores = model.evaluate(x_test, y_test, verbose=0)
print('IRNN test score:', scores[0])
print('IRNN test accuracy:', scores[1])

Epoch 1/200
1875/1875 [==============================] - 606s 323ms/step - loss: 2.2347 - accuracy: 0.1530 - val_loss: 2.0012 - val_accuracy: 0.2425
Epoch 2/200
1875/1875 [==============================] - 604s 322ms/step - loss: 1.9898 - accuracy: 0.2465 - val_loss: 1.9441 - val_accuracy: 0.2736
Epoch 3/200
1875/1875 [==============================] - 602s 321ms/step - loss: 1.9344 - accuracy: 0.2763 - val_loss: 1.8025 - val_accuracy: 0.3416
Epoch 4/200
1875/1875 [==============================] - 599s 320ms/step - loss: 1.7356 - accuracy: 0.3458 - val_loss: 1.5745 - val_accuracy: 0.4125
Epoch 5/200
1875/1875 [==============================] - 601s 321ms/step - loss: 1.5866 - accuracy: 0.4046 - val_loss: 1.5409 - val_accuracy: 0.4087
Epoch 6/200
1875/1875 [==============================] - 605s 323ms/step - loss: 1.5490 - accuracy: 0.4218 - val_loss: 1.5206 - val_accuracy: 0.4494
Epoch 7/200
1875/1875 [==============================] - 601s 320ms/step - loss: 1.5232 - accuracy: 0.4355